In [20]:
import sys, string
sys.path.append('/home/grg/git/xnat-monitor/')
from bbrc import xnat
from lxml import etree

central = xnat.connect_xnat('/home/grg/.xnat_bsc.cfg')
ns = {'xnat':"http://nrg.wustl.edu/xnat"}

In [31]:
subjects = [string.atoi(e) for e in open('/tmp/amyloid_subjects.txt').read().split('\n') if e != '']
# list of subjects with correct bvec/bval (but who failed the denoising step in my initial attempt
failed = [10162, 10515, 10627, 10649, 10711, 10949, 11262, 11514, 11745, 12279, 12741, 12991, 13035, 13105, 13107, 13193, 13259, 
             21136, 21138, 44008, 44301, 44396, 66008, 66027, 66100, 66162, 66301, 77012, 77164]
res = []
for subject in subjects:
    # Looks for a matching scan in ALFA_PLUS first, then ALFA_OPCIONAL
    e = xnat.subject_experiments(central, 'ALFA_PLUS', str(subject))
    if len(e) == 0:
        e = xnat.subject_experiments(central, 'ALFA_OPCIONAL', str(subject))
        
    doc = etree.fromstring(e[0].get())
    ans = doc.findall('.//xnat:date', ns)
    scandate = ans[0].text # date of acquisition
    ans = doc.findall('.//xnat:prearchivePath', ns)
    xnatdate = ans[0].text.split('/')[-2].split('_')[0] # date of upload to xnat
    res.append((subject, scandate, xnatdate, subject in failed))

# saving results in an Excel table and display the first lines
import pandas as pd
df = pd.DataFrame(res, columns=['subject', 'scandate', 'xnatdate', 'has_bvec'])
df.to_excel('/tmp/bvec_bval_dates.xls')
df.head()

,subject,scandate,xnatdate,has_bvec
0,10019,2017-02-15,20170331,False
1,10044,2017-06-27,20170627,False
2,10065,2016-10-21,20170330,False
3,10070,2017-06-13,20170621,False
4,10137,2017-06-07,20180117,False
